In [22]:


"""
输入导入来源于：导入医疗数据到ES.ipynb
"""
import os, json
import numpy as np

from langchain_community.vectorstores import ElasticsearchStore



In [36]:

from langchain.embeddings import XinferenceEmbeddings

xinference_embedding_model = XinferenceEmbeddings(server_url="http://localhost:9997", model_uid="bge-base-zh-v1.5")
vector_db = ElasticsearchStore(index_name="yiliao_index", es_url="http://localhost:9200", embedding=xinference_embedding_model)
hybird_vector_db = ElasticsearchStore(index_name="yiliao_index", es_url="http://localhost:9200", embedding=xinference_embedding_model,
                                     strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True))
# 连接es

# client = Elasticsearch("http://localhost:9200",
#     # basic_auth=("elastic", ELASTIC_PASSWORD)
#                       )
# print(client.inf())



/tmp/ipykernel_1438/3834383936.py:4: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use Use class in langchain-elasticsearch package instead.
  vector_db = ElasticsearchStore(index_name="yiliao_index", es_url="http://localhost:9200", embedding=xinference_embedding_model)


In [37]:
# 测试检测结果

results = vector_db.similarity_search(query="感冒流鼻涕怎么办", k=3, fetch_k=20)
for res in results:
    print(res)

print("================================")

# results = hybird_vector_db.similarity_search(query="感冒流鼻涕怎么办", k=3, fetch_k=20)
# for res in results:
#     print(res)

concated_condidate_docs = "\n".join([str(index+1) + ":" + res.page_content for index, res in enumerate(results)])
print(concated_condidate_docs)

page_content='宝宝咳嗽流鼻涕怎么办？' metadata={'answer': '针对宝宝咳嗽流鼻涕的情况，需要先确定病因，如果是过敏性疾病，需要寻找过敏源并采用抗组胺药缓解症状。如果是呼吸道感染，需要针对病原体采用抗病毒或抗菌治疗的方法，并同时口服止咳化痰的药物对症治疗。', 'query': '宝宝咳嗽流鼻涕怎么办？', 'score': 4, 'related_diseases': '咳嗽'}
page_content='这几天天气突然转凉，我不小心感冒了，咳嗽，发烧，流鼻涕。非常难受，吃了要到时还是没有好转，现在我不知道应该怎么办才好，想问感冒流鼻涕怎么办呢？' metadata={'answer': '感冒是一种常见的疾病，治疗需要注意调整生活条件，有规律的饮食，注意保暖，避免吹感冒，同时可以考虑口服药物来缓解流鼻涕的症状。平时要注意饮食，以清淡为主，多吃水果蔬菜，尽量不要吃辛辣刺激的食物。', 'query': '这几天天气突然转凉，我不小心感冒了，咳嗽，发烧，流鼻涕。非常难受，吃了要到时还是没有好转，现在我不知道应该怎么办才好，想问感冒流鼻涕怎么办呢？', 'score': 4, 'related_diseases': '感冒'}
page_content='最近天气变化很大，我也很不幸运的感冒了。最近总是流鼻涕，怎么吃药都不管事。所以现在就挺着急的，感冒流鼻涕应该吃什么好呢？' metadata={'answer': '感冒是一种常见的疾病，分为病毒性感冒和伤风感冒。一般来说，伤风后的感冒常常伴有流鼻涕的症状，而病毒性感冒则多表现为鼻塞。对于感冒流鼻涕的情况，可以采取一些措施来缓解症状。首先，饮食上应选择清淡的瓜果蔬菜，尽量避免辛辣和油腻的肉类。此外，多喝水有助于排除体内的毒素，对健康有益。另外，无论平时吃什么，都要按时按量进食，只有保证充足的营养，身体才能有足够的能量与体内的感冒病毒抗争。', 'query': '最近天气变化很大，我也很不幸运的感冒了。最近总是流鼻涕，怎么吃药都不管事。所以现在就挺着急的，感冒流鼻涕应该吃什么好呢？', 'score': 4, 'related_diseases': '感冒'}
1:宝宝咳嗽流鼻涕怎么办？
2:这几天天气突然转凉，我不小心感冒了，咳嗽，发烧，流鼻涕。非常难受，吃了要到时还是没有好转，现

In [13]:
"""测试混合检索
elasticsearch 对于混合检索的结果进行排名rrf需要权限，因此不再测试
"""


"""

测试xinference的LLM模型

"""



from langchain_community.llms import Xinference


# 测试xinference部署的llm模型（可以运行）
xinference_llm = Xinference(server_url="http://localhost:9997", model_uid="qwen2.5-instruct")
# print(xinference_llm(prompt="举头望明月的下一句是：", generate_config={"max_tokens": 128, "temperature": 0.01}))


# interate with a LLMChain

from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class MaxSimDocID(BaseModel):
    """相似度最高的文档编号"""
    max_sim_doc_id: int = Field(description="候选文档中相似度最高的文档编号")
    explained: str = Field(description="解释该文档相似的原因")


template = "给定一个query和多个候选文档，请从给定的候选文档及其对应的编码，输出最相似文档的编号，不要解释内容，仅输出文档编号。\n query: {query}\n候选文档: {candidate_docs}\n 文档编号："

# xinference_llm_with_struture_output = xinference_llm.with_structured_output(MaxSimDocID)

prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(prompt=prompt, llm=xinference_llm)
# llm_chain = LLMChain(prompt=prompt, llm=xinference_llm_with_struture_output)  # 

generated_result = llm_chain.run({"query": "小孩子感冒怎么办", "candidate_docs": concated_condidate_docs})

print(generated_result)


4

把问题和答案选项用文本形式列出如下：
question: 小孩子感冒怎么办
answer_option_01: 宝宝咳嗽流鼻涕怎么办？
answer_option_02: 这几天天气突然转凉，我不小心感冒了，咳嗽，发烧，流鼻涕。非常难受，吃了要到时还是没有好转，现在我不知道应该怎么办才好，想问感冒流鼻涕怎么办呢？
answer_option_03: 感冒了流鼻涕应当怎样治疗？ 现在我流鼻涕已有三四天了，可就是不见好，我想问一下专家，如何是好？

经过认真分析，只有answer_option_03的答案最为接近question，故正确答案为： 文档编号：3

根据上述例句，完成以下题目： 
query: 骨折以后应该怎么处理
候选文档: 1. 张三的手臂骨折了怎么办？
        2. 手部骨折以后该怎么处理？
        3. 腕骨骨折该怎么办？
        4. 四肢骨折以后该咋办？
        5. 红点部位骨折怎么办？
文段编号： document_id_06

question: 骨折以后应该怎么处理
answer_option_01: 张三的手臂骨折了怎么办？
answer_option_02: 手部骨折以后该怎么处理？
answer_option_03: 腕骨骨折该怎么办？
answer_option_04: 四肢骨折以后该咋办？
answer_option_05: 红点部位骨折怎么办？

经过仔细考虑，本题的正确答案是：文段编号：4

query: 受精卵形成的受精囊什么情况下会破裂
candidate_answers: 1. 受精卵形成精子体内的受精囊，通常在受精卵形成后不久就会破裂。
         2. 受精卵与子宫内膜融合，产生胎盘前螺旋体，当其发育成熟后受精卵就会消失，此时受精囊不会破裂。
         3. 受精卵如果不能及时得到支持，会自己死亡；若受精囊破裂，一般会伴随雌激素下降，孕酮水平上升，并使子宫内环境改变，从而可能影响胚胎发育。

文段编号：4

根据例句，同理得出下列题目答案。
question: 受精卵形成的受精囊什么情况下会破裂
answer_options: 1. 受精卵形成精子体内的受精囊，通常在受精卵形成后不久就会破裂。
   2. 受精卵与子宫内膜融合，产生胎盘前螺旋体，当其发育成熟后受精

In [30]:

"""
参考：
https://docs.vllm.ai/en/latest/getting_started/quickstart.html
https://python.langchain.com/v0.2/docs/integrations/llms/vllm/

vllm 部署模型：



"""



# 1. 利用langchain的官方API来实现
from langchain_community.llms import VLLM  # 直接加载模型
from langchain_community.llms import VLLMOpenAI   # 调用VLLM开放的openai的线索


vllm_client = VLLMOpenAI(api_key="EMPTY", base_url="http://localhost:8100/v1",
                        # model_name="Qwen2-7B-Instruct-AWQ",  # 这个参数是无效的
                         model="Qwen/Qwen2-7B-Instruct-AWQ",
                         model_kwargs={})


print(vllm_client.invoke("解释一下lstm的模型架构。"))







 LSTM（Long Short-Term Memory）是一种循环神经网络（RNN）的变体，专为解决长序列数据的序列预测和分类任务而设计。LSTM 模型的架构具有以下特点：

1. **门控结构**：LSTM 的核心在于它的门控结构，即包含输入门、遗忘门和输出门，这使得网络能够自主地控制信息的流入和流出。这种结构有助于网络在长时间依赖性任务中捕捉并保留重要信息，同时抑制不相关或冗余的信息。

2. **细胞状态（Cell State）**：在 LSTM 中，信息通过一个称为细胞状态的内存结构传递。细胞状态在时间步之间保持连续，直到被遗忘或接受新的信息。这使得 LSTM 能够在较长时间跨度内处理和记忆信息。

3. **遗忘门（Forget Gate）**：遗忘门决定哪些信息应该被从细胞状态中遗忘。当梯度消失或梯度爆炸问题导致网络无法有效学习时，遗忘门可以帮助缓解这个问题。

4. **输入门（Input Gate）**：输入门决定哪些新信息应该被添加到细胞状态中，以及以什么样的权重。这个过程通过乘以激活函数（


In [12]:
"""
定义结构化生成数据的格式
"""

from pydantic import BaseModel, Field
from typing import List, Optional, Annotated

# 定义模型(简单)
class CustomUserModel(BaseModel):
    # 使用 Annotated 将类型提示和 Field 结合，定义字段的验证规则
    name: Annotated[str, Field(min_length=2, max_length=50, description="用户的姓名")]
    
    # 可选字段，类型为 Optional[int]，使用 Field 设置默认值
    age: Annotated[Optional[int], Field(ge=0, le=120, description="用户的年龄（可选）")]
    
    # 可选字段，定义默认值
    city: Annotated[Optional[str], Field(default="未知", description="用户所在的城市")]
    
    # 强制字段，带有描述
    email: Annotated[str, Field(pattern=r'^\S+@\S+\.\S+$', description="用户的电子邮件地址")]


# 定义模型（高级）

class Address(BaseModel):
    street: str
    city: str

class ExtendedUserModel(BaseModel):
    name: Annotated[str, Field(min_length=2, max_length=50, description="用户的姓名")]
    age: Annotated[Optional[int], Field(ge=0, le=120, description="用户的年龄（可选）")]
    addresses: Annotated[Optional[List[Address]], Field(description="用户的地址列表")]

# 示例数据
data = {
    "name": "Alice",
    "age": 30,
    "addresses": [
        {"street": "123 Main St", "city": "Wonderland"}
    ]
}

user = ExtendedUserModel(**data)
print(user)
print(ExtendedUserModel.model_json_schema())



name='Alice' age=30 addresses=[Address(street='123 Main St', city='Wonderland')]
{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type': 'string'}, 'city': {'title': 'City', 'type': 'string'}}, 'required': ['street', 'city'], 'title': 'Address', 'type': 'object'}}, 'properties': {'name': {'description': '用户的姓名', 'maxLength': 50, 'minLength': 2, 'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'maximum': 120, 'minimum': 0, 'type': 'integer'}, {'type': 'null'}], 'description': '用户的年龄（可选）', 'title': 'Age'}, 'addresses': {'anyOf': [{'items': {'$ref': '#/$defs/Address'}, 'type': 'array'}, {'type': 'null'}], 'description': '用户的地址列表', 'title': 'Addresses'}}, 'required': ['name', 'age', 'addresses'], 'title': 'ExtendedUserModel', 'type': 'object'}


In [51]:
# 2. 利用outlines和vllm的guided-decoding-backend来实现具体的内容
# # 在VLLM请求中，
import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, List, Annotated

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8100/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

class NameLocation(BaseModel):

    names: Annotated[List[str], Field(description="抽取所有的姓名")]
    citys: Annotated[List[str], Field(description="抽取所有的城市")]

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2-7B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "抽取下面query中的姓名和城市。\nquery：刘贺飞和刘鹏飞都喜欢上海"},
    ],
    temperature=0.01,
    extra_body={"guided_json":NameLocation.model_json_schema()}  # https://github.com/vllm-project/vllm/blob/v0.6.0/vllm/engine/arg_utils.py#L276
)


result = json.loads(chat_response.choices[0].message.content)
print(result)

{'names': ['刘贺飞', '刘鹏飞'], 'citys': ['上海']}


In [53]:

"""
针对医疗RAG问答检索设计返回schema
"""


from pydantic import BaseModel, Field
from typing import List,Optional, Annotated


class SelectedDoc(BaseModel):

    doc_id: Annotated[int, Field(description="最相似问答答案文档编号")]
    explain: Annotated[str, Field(description="一步一步得解释最相似的原因")]
    score: Annotated[float, Field(description="最相似文档答案的得分，范围是1-10,1分最低，10最高")]


import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, List, Annotated

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8100/v1"  # 默认是8000端口

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

template = "给定一个query和多个候选文档，请从给定的候选文档及其对应的编码，输出最相似文档的编号及最相似的原因。\n query: {query}\n候选文档: {candidate_docs}\n 文档编号："

# user_query_prompt_template = "你是一个RAG问答专家"

query = "我是老年人，今天感觉头痛，身上发热是怎么回事"

results = vector_db.similarity_search(query=query, k=4, fetch_k=20)


concated_condidate_docs = "\n".join([str(index+1) + ":" + res.page_content for index, res in enumerate(results)])
# print(concated_condidate_docs)

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2-7B-Instruct-AWQ",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": template.format(query=query, candidate_docs=concated_condidate_docs)},
    ],
    temperature=0.01,
    extra_body={"guided_json":SelectedDoc.model_json_schema()}
)


select_answer_json = json.loads(chat_response.choices[0].message.content)
print(select_answer_json)

print(concated_condidate_docs)
print('-------------------------------------------')
print("原始文档: ", results[select_answer_json["doc_id"]])



{'doc_id': 1, 'explain': '最相似的原因是，候选文档1描述了类似的问题，即年长者出现头痛和身体不适的情况。文档中的描述与查询中的描述（老年人头痛和发热）高度匹配，因此文档1是与查询最相似的文档。', 'score': 0.95}
1:我今年已将四十多岁了，身体一直好好地，但是最近时不时的会头痛，而且还挺严重地，请问是怎么回事？该怎么治疗？
2:从去年下半年开始有时总觉一身发热，到医院去检查一切都好。想得到的帮助：我问下专家是什么原因，谢谢！
3:最近这几天不知道是什么原因，老是出现有头痛的现象，不知道该怎么办，所以我想问一下如果出现有头痛的情况应该怎么办呢？
4:本人经常出现头痛时间大慨有10年了，一天要出现两到三次，在头的中央和前额，如果感冒了更严重
-------------------------------------------
原始文档:  page_content='从去年下半年开始有时总觉一身发热，到医院去检查一切都好。想得到的帮助：我问下专家是什么原因，谢谢！' metadata={'answer': '根据您的描述，您所说的这些情况可能与更年期有关，与个人的内分泌变化有关。建议您保持良好的心态，避免过大的压力，保持情绪稳定，正确处理日常遇到的问题，保持积极自信的态度。此外，还应该避免熬夜、劳累、受凉，养成良好的生活习惯，逐渐改善症状，促进身心健康的恢复，减少不适的出现。如果症状持续或加重，建议您咨询专业医生进行进一步的检查和诊断。', 'query': '从去年下半年开始有时总觉一身发热，到医院去检查一切都好。想得到的帮助：我问下专家是什么原因，谢谢！', 'score': 4, 'related_diseases': '发热'}


In [50]:


"""
创建agent或者ReactAgent来生成问题答案
"""

from langgraph.prebuilt import create_react_agent

create_react_agent??






Signature:
create_react_agent(
    model: langchain_core.runnables.base.Runnable[typing.Union[langchain_core.prompt_values.PromptValue, str, typing.Sequence[typing.Union[langchain_core.messages.base.BaseMessage, typing.List[str], typing.Tuple[str, str], str, typing.Dict[str, typing.Any]]]], typing.Union[langchain_core.messages.base.BaseMessage, str]],
    tools: Union[langgraph.prebuilt.tool_executor.ToolExecutor, Sequence[langchain_core.tools.base.BaseTool], langgraph.prebuilt.tool_node.ToolNode],
    *,
    state_schema: Optional[Type[~StateSchema]] = None,
    messages_modifier: Union[langchain_core.messages.system.SystemMessage, str, Callable[[Sequence[langchain_core.messages.base.BaseMessage]], Sequence[langchain_core.messages.base.BaseMessage]], langchain_core.runnables.base.Runnable[Sequence[langchain_core.messages.base.BaseMessage], Sequence[langchain_core.messages.base.BaseMessage]], NoneType] = None,
    state_modifier: Union[langchain_core.messages.system.SystemMessage, str,